In [1]:
import pandas as pd
# anno_1 = '../Human_anno/humananno_res/lsy_scene.xlsx'
# anno_2 = '../Human_anno/humananno_res/lsy_scene.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','scene'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','scene'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

In [3]:
dimension = 'color'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(length):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/length
# gptvsanno2_spearman = 3*gptvsanno2_spearman/length
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/length
# gptvsannomean = 3*gptvsannomean/length
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [6]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
idexls = []
for i in range(0,len(oc)):
    idexls.append(i)
length = len(idexls)

In [7]:
gptvsanno1_spearman = np.zeros(length)
gptvsanno2_spearman = np.zeros(length)
anno1vsanno2_spearman = np.zeros(length)
gptvsannomean_spearman = np.zeros(length)
baselinevsanno1_spearman = np.zeros(length)
baselinevsanno2_spearman = np.zeros(length)
baselinevsannomean_spearman = np.zeros(length)

gptscore = np.zeros([len(models)])
anno1score = np.zeros([len(models)])
anno2score = np.zeros([len(models)])
annomeanscore =np.zeros([len(models)])
# baseline_rank = np.zeros([7])
badeval = []

for i in idexls:
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    # baseline_rank = np.array(list(oc[i]['baseline_rank'].values())[3:])

    anno1 = human_anno[:,2]
    # anno2 = human_anno[:,1]
    anno2 = human_anno[:,3]

    # gpt4o_eval_rs[gpt4o_eval_rs == 4] = 5
    # anno1[anno1 == 4] = 5
    # anno2[anno2 == 4] = 5

    annomean = (anno1 + anno2)/2


    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    annomeanscore += annomean


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2 = calculate_spearman_manual(anno1,anno2)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    # baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
    # baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
    # baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)

    # j = int((i-108)/3)
    j = i
    gptvsanno1_spearman[j] = gptvsanno1
    gptvsanno2_spearman[j] = gptvsanno2
    anno1vsanno2_spearman[j] = anno1vsanno2
    gptvsannomean_spearman[j] = gptvsannomean
    # baselinevsanno1_spearman[i] = baselinevsanno1
    # baselinevsanno2_spearman[i] = baselinevsanno2
    # baselinevsannomean_spearman[i] = baselinevsannomean

    if gptvsanno2 <0.2:
        badeval.append(i)

gptscore = gptscore/length
anno1score = anno1score/length
anno2score = anno2score/length
annomeanscore = annomeanscore/length

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.mean())
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
# print("Baseline vs Anno1 Spearman: ",baselinevsanno1_spearman.mean())
# print("Baseline vs Anno2 Spearman: ",baselinevsanno2_spearman.mean())
# print("Baseline vs AnnoMean Spearman: ",baselinevsannomean_spearman.mean())


print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("AnnoMean average score: ",annomeanscore)

GPT vs Anno1 Spearman:  0.5415616246498599
GPT vs Anno2 Spearman:  0.6035714285714285
Anno1 vs Anno2 Spearman:  0.6318277310924371
GPT vs AnnoMean Spearman:  0.5774509803921568
GPT average score:  [4.3372549  4.45490196 4.64313725 4.69411765 3.48627451 4.28235294
 3.90588235]
Anno1 average score:  [2.8745098  3.97254902 3.89411765 2.73333333 1.8627451  2.55294118
 2.41568627]
Anno2 average score:  [3.23137255 3.28235294 3.39215686 2.90980392 2.02745098 2.70980392
 2.17254902]
AnnoMean average score:  [3.05294118 3.62745098 3.64313725 2.82156863 1.94509804 2.63137255
 2.29411765]


In [8]:
from collections import Counter
badeval_prompt = []
for i in badeval:
    # print("spearman",gptvsannomean_spearman[i])
# print("videos",oc[i]['videos']['gen2'])
    # print("prompt",oc[i]['prompt_en'])
    badeval_prompt.append(oc[i]['prompt_en'])
    print(i%3,i,oc[i]['prompt_en'],gptvsanno2_spearman[i])
    # print("GPT score",oc[i]['gpt4o_eval'])
    # print("Anno score",oc[i]['human_anno'])
    # print("GPT reasons",gpt4o_eval_history[str(i)])

2 29 A red car 0.1428571428571429
1 55 A red bird 0.1160714285714286
0 66 An orange bird 0.0982142857142857
1 70 A purple bird -0.0714285714285714
1 73 A pink bird 0.1517857142857143
0 81 A black cat -0.1339285714285714
0 84 A white cat 0.1875
2 86 A white cat 0.1517857142857143
0 87 An orange cat 0.0535714285714286
2 92 A yellow cat 0.1964285714285714
1 94 A red umbrella -0.02678571428571419
0 147 A red bowl 0.1964285714285714
1 154 A blue bowl 0.1785714285714286
0 165 A pink bowl 0.1964285714285714
2 170 A black bowl 0.1517857142857143
1 181 A blue chair 0.0535714285714286
1 196 A black chair 0.1160714285714286
2 215 An orange clock 0.1339285714285714
2 218 A purple clock 0.1071428571428571
2 224 A black clock 0.125
0 225 A white clock -0.008928571428571397


In [9]:
conuter = Counter(badeval_prompt)
repeated = {k:v for k,v in conuter.items() if v>1}

In [10]:
# print(sum(repeated.values()))
# print(repeated)
# with open('D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\BadEval4OverallConsistency.json','w') as f:
#     json.dump(repeated,f,indent=4)